## 为什么要有这个番外
修改开源代码（仅在CPU）执行时，希望能全挪到 GPU 上加速。
这里面也有门道的，PyTorch 要求输入数据与模型（所维护的参数）都要放在同一个 GPU 上，
否则就报错：
> RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)

In [5]:
import time

import torch
from torch import nn

### 1.数据通过 torch.Tensor.cuda() 放到GPU 上

#### 注意点
1. 如果定义了 `DataLoader`，返回的数据时应当是 `.to(device)` 后的
2. 如果定义 `DataLoader` 时声明了 `collate_fn` ， `.to(device)`操作应当放在 `collate_fn` 内

In [8]:
# 准备数据
n = 2 << 10  # 样本数量

X = 10 * torch.rand([n, 2]) - 5.0  # torch.rand是均匀分布
w0 = torch.tensor([[2.0, -3.0]])
b0 = torch.tensor([[10.0]])
Y = X @ w0.t() + b0 + torch.normal(0.0, 2.0, size=[n, 1])  # @表示矩阵乘法，增加正态扰动

# 移动到GPU上
print('torch.cuda.is_available()=', torch.cuda.is_available())
X = X.cuda() # 转移到GPU
Y = Y.cuda()

torch.cuda.is_available()= True


## 2. 模型实例用 torch.nn.Module.to() 搬运到 GPU

#### 注意点
1. 如果存在 sub 模型和下游模型，sub 模型实例和下游模型实例都要执行`.to(device)`操作


In [9]:
# 定义模型
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.randn_like(w0))
        self.b = nn.Parameter(torch.zeros_like(b0))

    # 正向传播
    def forward(self, x):
        return x @ self.w.t() + self.b


linear = LinearRegression()

# 移动模型到GPU上
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
linear.to(device)

LinearRegression()

## 3.训练

In [10]:
optimizer = torch.optim.Adam(linear.parameters(), lr=0.1)
l = nn.MSELoss()

def train(epochs):
    tic = time.time()
    for epoch in range(epochs):
        optimizer.zero_grad()
        Y_pred = linear(X)
        loss = l(Y_pred, Y)
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0:
            print({'epoch': epoch, 'loss': loss.item()})
    toc = time.time()
    print('time used:', toc - tic)


train(500)

{'epoch': 0, 'loss': 225.893310546875}
{'epoch': 50, 'loss': 33.50811004638672}
{'epoch': 100, 'loss': 8.915048599243164}
{'epoch': 150, 'loss': 4.357623100280762}
{'epoch': 200, 'loss': 3.8925089836120605}
{'epoch': 250, 'loss': 3.8690719604492188}
{'epoch': 300, 'loss': 3.8685646057128906}
{'epoch': 350, 'loss': 3.868561267852783}
{'epoch': 400, 'loss': 3.868561267852783}
{'epoch': 450, 'loss': 3.868561267852783}
time used: 3.5719974040985107
